In [1]:
import os
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
years  = ["2016_3082","2017_3083","2018_3084","2019_3085","2020_3086","2021_3087"]

folderheader = "5841834/Kelmarsh_SCADA_"
turbine_data_header = "/Turbine_Data_Kelmarsh"
status_header = "/Status_Kelmarsh"

In [3]:
years  = ["2016_3082","2017_3083","2018_3084","2019_3085","2020_3086","2021_3087"]

folderheader = "5841834/Kelmarsh_SCADA_"
turbine_data_header = "/Turbine_Data_Kelmarsh"
status_header = "/Status_Kelmarsh"

def process_files(years, folderheader, keyword):
    dataframes = []
    for year in years:
        # List all files in the directory for the given year
        files = os.listdir(folderheader + year)
        # Filter files based on the keyword
        filtered_files = [file for file in files if keyword in file]
        
        for filename in filtered_files:
            # Extract the turbine number from the filename
            turbine_number = filename.split('_')[-5]
            # Construct the full file path
            filepath = os.path.join(folderheader+year, filename)
            # Read the CSV file
            df = pd.read_csv(filepath, header=9)
            # Add a column for the turbine number
            df['Turbine'] = turbine_number
            # Append the dataframe to the list
            dataframes.append(df)
    
    # Concatenate all dataframes into one and reset the index
    return pd.concat(dataframes, ignore_index=True)

# Usage

status_df = process_files(years, folderheader, 'Status')
turbine_df = process_files(years, folderheader, 'Turbine')

In [4]:
status_df['Timestamp start'] = pd.to_datetime(status_df['Timestamp start'])
turbine_df["# Date and time"] = pd.to_datetime(turbine_df["# Date and time"])


In [5]:
error_cats = ['Emergency stop top box', 'Emergency stop nacelle',
       'Safety chain open', 'UPS buffer time too short',
       'Emergency stop base box', 'Manual stop - remote',
       'Oscillation encoder tower', 
       'Frequency converter error', 'Repeating error BP52',
       'WEC shut down', 'Yaw speed high', 'Yaw velocity too low',
       'Supply circuit breaker off-state',
       'Frequency converter not ready', 'Overload generator fan 1',
       'Overload yaw motor 2&4', 
       'Overload generator fan 2', 'Overload generator fan 3',
       'No speed development', 'Pitch angle deviation', 'Yaw error',
       'Pitch run-away (hub box v.>=4)', 'Maximum grid frequency',
       'PLC hardware error', 'Overload gear oil pump',
       'Max. cable twistangle',
       'Uncontrolled yaw movement', 'Limit switch error 95° axis 1',
       'Low gearbox oil pressure', 'Feedback brake 1',
       'High rotor speed nacelle', 'Implausible gear speed',
       'Low hydraulic pressure', 
       'Rotor sensor A defective', 'Overload yaw motor 1&3',
       'Disc filter adaption implausible', 'Vane defect',
       'Tower resonance', 'Overload fan oil cooler gear',
       'Rotor overspeed nacelle', 'Batt. undervoltage/overvoltage',
       'Service generator brushes', 'Battery voltage axis 1',
       'Error pitch converter 2']

In [6]:
# Function to preprocess and correct the duration strings
def preprocess_duration(duration):
    # Check if the duration is negative
    is_negative = duration.startswith('-')
    
    # Remove all negative signs and split by colon
    parts = duration.replace('-', '').split(':')
    
    # Construct the correct timedelta string
    if is_negative:
        # Apply negative sign at the beginning only
        duration = f"-{parts[0]} hours {parts[1]} minutes {parts[2]} seconds"
    else:
        duration = f"{parts[0]} hours {parts[1]} minutes {parts[2]} seconds"
    
    return duration

# Apply the preprocessing function and convert to timedelta

status_reduced = status_df[status_df["Duration"] != "-"]
status_reduced['clean_duration'] = status_reduced['Duration'].apply(preprocess_duration)


/var/folders/4y/3ynvczrn5fldxkh3r7m_fh8r0000gn/T/ipykernel_66474/1390822933.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  status_reduced['clean_duration'] = status_reduced['Duration'].apply(preprocess_duration)


In [13]:
relevant_errors = ["Pitch run-away (hub box v.>=4)",
"Frequency converter error",
"Overload generator fan 1", "Overload generator fan 2", "Overload generator fan 3"]


In [14]:
error_labels = status_df[status_df["Message"].isin(relevant_errors)].loc[:,["Timestamp start","Turbine","Message"]].reset_index()

In [21]:
error_labels["date"] = error_labels["Timestamp start"].dt.date

error_labels = error_labels.drop_duplicates(subset=['Turbine','Message','date'])

In [22]:
error_labels

,index,Timestamp start,Turbine,Message,date
0,70,2016-01-29 10:08:33,2,Frequency converter error,2016-01-29
1,77,2016-01-30 07:33:38,2,Frequency converter error,2016-01-30
2,81,2016-02-01 18:43:27,2,Frequency converter error,2016-02-01
3,83,2016-02-02 16:04:36,2,Frequency converter error,2016-02-02
5,293,2016-03-15 12:51:40,2,Overload generator fan 1,2016-03-15
...,...,...,...,...,...
385,271763,2021-01-19 10:38:48,5,Overload generator fan 2,2021-01-19
388,280499,2021-05-15 06:55:24,4,Overload generator fan 1,2021-05-15
389,280603,2021-05-16 07:59:48,4,Overload generator fan 2,2021-05-16
390,280604,2021-05-16 07:59:49,4,Overload generator fan 3,2021-05-16


In [43]:
df = error_labels
# Assuming 'df' is your DataFrame
# First, make sure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Sort the DataFrame by 'Turbine' and 'date'
df = df.sort_values(by=['Turbine', 'date', 'Message'])


# Check for consecutive messages for the same turbine on consecutive days
df['duplicate'] = df.duplicated(subset=['Turbine', 'Message'], keep='first') & \
                  (df['date'] - df['date'].shift() == pd.Timedelta(days=1)) & \
                  (df['Turbine'] == df['Turbine'].shift())



/var/folders/4y/3ynvczrn5fldxkh3r7m_fh8r0000gn/T/ipykernel_66474/2195948553.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [46]:
# Keep only the non-duplicate entries
df = df[~df['duplicate']]

# Drop the 'duplicate' column as it's no longer needed
df.drop(columns=['duplicate'], inplace=True)

# Reset the index of the DataFrame if needed
df.reset_index(drop=True, inplace=True)

/var/folders/4y/3ynvczrn5fldxkh3r7m_fh8r0000gn/T/ipykernel_66474/1263209646.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['duplicate'], inplace=True)


In [47]:
df

,index,Timestamp start,Turbine,Message,date
0,9976,2016-01-14 19:28:05,1,Overload generator fan 1,2016-01-14
1,9974,2016-01-14 19:28:05,1,Overload generator fan 2,2016-01-14
2,9975,2016-01-14 19:28:05,1,Overload generator fan 3,2016-01-14
3,10012,2016-01-24 16:51:17,1,Frequency converter error,2016-01-24
4,10157,2016-03-01 18:35:40,1,Frequency converter error,2016-03-01
5,10563,2016-04-25 12:30:50,1,Overload generator fan 1,2016-04-25
6,10561,2016-04-25 12:30:49,1,Overload generator fan 2,2016-04-25
7,10562,2016-04-25 12:30:49,1,Overload generator fan 3,2016-04-25
8,10606,2016-04-26 09:43:14,1,Overload generator fan 2,2016-04-26
9,10607,2016-04-26 09:43:14,1,Overload generator fan 3,2016-04-26


In [49]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Now when you display the DataFrame, it will show all rows and columns
df

,index,Timestamp start,Turbine,Message,date
0,9976,2016-01-14 19:28:05,1,Overload generator fan 1,2016-01-14
1,9974,2016-01-14 19:28:05,1,Overload generator fan 2,2016-01-14
2,9975,2016-01-14 19:28:05,1,Overload generator fan 3,2016-01-14
3,10012,2016-01-24 16:51:17,1,Frequency converter error,2016-01-24
4,10157,2016-03-01 18:35:40,1,Frequency converter error,2016-03-01
5,10563,2016-04-25 12:30:50,1,Overload generator fan 1,2016-04-25
6,10561,2016-04-25 12:30:49,1,Overload generator fan 2,2016-04-25
7,10562,2016-04-25 12:30:49,1,Overload generator fan 3,2016-04-25
8,10606,2016-04-26 09:43:14,1,Overload generator fan 2,2016-04-26
9,10607,2016-04-26 09:43:14,1,Overload generator fan 3,2016-04-26


In [50]:
df.to_csv("relevant_errors.csv")

In [38]:


# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')